In [ ]:
#!7za -y x "/content/drive/MyDrive/Data_Science_Project/Module_Project_CNN/data/image"

In [ ]:

#!cp -r ./origin "/content/drive/MyDrive/Data_Science_Project/Module_Project_CNN/origin.7z*"

In [1]:
label_file_path=r"/content/drive/MyDrive/Data_Science_Project/Module_Project_CNN/data/label/label.lst"
images_folder_path=r"/content/drive/MyDrive/Data_Science_Project/Module_Project_CNN/origin.7z*"

In [2]:
import pandas as pd
df_info=pd.read_csv(label_file_path, sep=" ",header=None)
df_info.columns=['image_name','face_id_in_image','face_box_top','face_box_left','face_box_right','face_box_bottom','face_box_cofidence','expression_label']
df_info.head()

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_cofidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0


In [3]:
df_info.shape

(91793, 8)

In [ ]:
df_info.value_counts()

image_name                  face_id_in_image  face_box_top  face_box_left  face_box_right  face_box_bottom  face_box_cofidence  expression_label
awe_teacher_108.jpg         1                 600           1524           1572            648              72.4882             3                   37
astound_expression_400.jpg  3                 288           308            388             368              72.5160             3                   16
awe_student_400.jpg         2                 664           388            436             712              82.8833             6                   13
amazed_chinese_563.jpg      3                 652           1908           1956            700              56.7663             6                   12
                            1                 176           576            624             224              57.5098             6                   12
                                                                                                    

In [5]:
df_sel=df_info[df_info.face_box_cofidence > 80]
#print(df_sel)
len(df_sel)


12127

In [ ]:
df_sel.shape

(12127, 8)

In [6]:
import cv2
import os
import concurrent.futures

x = []
y = []

for i in range(len(df_sel)):
    img_name = row["image_name"]
    x1 = row["face_box_left"]
    y1 = row["face_box_top"]
    x2 = row["face_box_right"]
    y2 = row["face_box_bottom"]
    label = row["expression_label"]
    img_path = os.path.join(images_folder_path, img_name)

    image = cv2.imread(img_path)

    if image is not None:
        cropped = image[y1:y2, x1:x2]
   
    if cropped is not None:
        resized_face = cv2.resize(cropped, (64, 64))
   

    return resized_face, label




In [7]:
len(x)

8847

In [8]:
#Data preparation
import numpy as np
X= np.array(x)
y= np.array(y)


In [12]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical



# Normalization
X_normalized = X / 255.0

# Sampling 
sample_size = 3500
random_indices = np.random.choice(len(X_normalized), size=sample_size )
X_sampled = X_normalized[random_indices]
Y_sampled = y[random_indices]

#Encoding
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_sampled)

# Train-Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X_sampled, Y_encoded, test_size=0.2, random_state=42)

# Load and Fine-tune VGG16 Model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(64,64,3))

# Freeze the layers so that they are not updated during training
for layer in vgg16.layers:
    layer.trainable = False

# Add custom layers on top of VGG16 for classification
x = Flatten()(vgg16.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

# Create the model
model = Model(inputs=vgg16.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test, Y_test))

# Calculate accuracy on test data
loss, accuracy = model.evaluate(X_test, Y_test, batch_size=32)
print(f"Test accuracy: {accuracy:.2f}")


Epoch 1/10
88/88 [==============================] - 5s 39ms/step - loss: 1.2033 - accuracy: 0.5889 - val_loss: 1.0692 - val_accuracy: 0.6300
Epoch 2/10
88/88 [==============================] - 2s 19ms/step - loss: 0.9965 - accuracy: 0.6504 - val_loss: 1.0884 - val_accuracy: 0.6371
Epoch 3/10
88/88 [==============================] - 2s 23ms/step - loss: 0.8912 - accuracy: 0.6911 - val_loss: 0.9895 - val_accuracy: 0.6500
Epoch 4/10
88/88 [==============================] - 2s 19ms/step - loss: 0.7908 - accuracy: 0.7232 - val_loss: 0.9357 - val_accuracy: 0.6600
Epoch 5/10
88/88 [==============================] - 2s 24ms/step - loss: 0.6655 - accuracy: 0.7682 - val_loss: 0.9306 - val_accuracy: 0.6800
Epoch 6/10
88/88 [==============================] - 2s 24ms/step - loss: 0.5709 - accuracy: 0.8068 - val_loss: 1.0371 - val_accuracy: 0.6714
Epoch 7/10
88/88 [==============================] - 2s 19ms/step - loss: 0.5045 - accuracy: 0.8382 - val_loss: 1.0223 - val_accuracy: 0.6814
Epoch 8/10
88